In [1]:
import os
from itertools import product
import numpy as np

In [2]:
old_ets = np.array([4])
old_etas = np.array([0])
et_key = 'L2Calo_et'
eta_key = 'L2Calo_eta'
dataset = 'data17_13TeV.AllPeriods.sgn.probes_lhmedium_EGAM1.bkg.VProbes_EGAM7.GRL_v97'
#homepath = os.path.expanduser('~')
homepath = 'E:\\data'
datapath = os.path.join(homepath, dataset)
filepath = os.path.join(datapath, dataset + '_et{et}_eta{eta}.npz')

In [3]:
new_etbins = np.array([15, 20, 30, 40, 50, 100, 1e6])   #Gev
new_etabins = np.array([0., 0.8, 1.37, 1.54, 2.37, 2.5])
new_ets = np.arange(len(new_etbins)+1)
new_dataset = 'new_bin_' + dataset
new_datapath = os.path.join(homepath, new_dataset)
new_filepath = os.path.join(new_datapath, new_dataset + '_et{et}_eta{eta}.npz')
if not os.path.exists(new_datapath):
    os.makedirs(new_datapath)

In [4]:
for old_et, old_eta in product(old_ets, old_etas):
    print(f'Reading et {old_et} eta {old_eta}')
    data = dict(np.load(filepath.format(et=old_et, eta=old_eta)))
    et_data_idx = np.where(data['features'] == et_key)[0][0]
    eta_data_idx = np.where(data['features'] == eta_key)[0][0]

    if old_et == 4:
        for new_et, new_eta in product(np.array([4,5]), np.array([old_eta])):
            if new_et == 4:
                limit = new_etbins[new_et+1] * 1000     #Mev
                selector = data['data'][:, et_data_idx] < limit
            else:
                limit = new_etbins[new_et] * 1000       #Mev
                selector = data['data'][:, et_data_idx] >= limit
            new_data = {
                'features': data['features'],
                'etBins': new_etbins,
                'etaBins': new_etabins,
                'etBinIdx': new_et,
                'etaBinIdx': new_eta,
                'data': data['data'][selector].copy(),
                'target': data['target'][selector].copy()
            }
            print(f'Dumping et {new_et} eta {new_eta} with {limit} Mev')
            np.savez(new_filepath.format(et=new_et, eta=new_eta), **new_data)
    else:
        print(f'Dumping et {old_et} eta {old_eta}')
        new_data['etBins'] = new_etbins
        np.savez(new_filepath.format(et=old_et, eta=old_eta))



Reading et 4 eta 0
Dumping et 4 eta 0 with 100000.0 Mev
Dumping et 5 eta 0 with 100.0 Mev


In [5]:
data.keys()

dict_keys(['features', 'etBins', 'etaBins', 'etBinIdx', 'etaBinIdx', 'data', 'target'])

In [6]:
data['etBins']

array([1.5e+01, 2.0e+01, 3.0e+01, 4.0e+01, 5.0e+01, 1.0e+06])